# Formula 1 Radio Sentiment Analysis with Bert

In this notebook, two BERT models are implemented:

1. First one is a **trained BERT from sratch**.
2. Secondly, a **pre-trained BERT model** will be downloaded and fine-tuned with my team radios.